# Otras Clases en Kotlin
Exploramos tipos especiales de clases: enums, sealed classes y sealed interfaces usando ejemplos de HTTP.

## Enum Classes (Enumeraciones)
Las enums representan un conjunto fijo de constantes, ideales para valores que no cambian.

In [1]:
// Enum básico para códigos HTTP
enum class HttpStatus {
    OK,
    NOT_FOUND,
    SERVER_ERROR
}

// Usar el enum
val status = HttpStatus.OK
println("Estado: $status")
println("Nombre: ${status.name}")
println("Posición: ${status.ordinal}")

Estado: OK
Nombre: OK
Posición: 0
200


In [2]:
// Enum con propiedades - Códigos de error HTTP
enum class HttpErrorEnum(val code: Int, val message: String) {
    Unauthorized(401, "No tienes permisos"),
    NotFound(404, "Recurso no encontrado"),
    ServerUnavailable(503, "Servicio no disponible");
    
    // Función dentro del enum
    fun getFullMessage(): String {
        return "Error $code: $message"
    }
}

// Usar enum con propiedades
val error = HttpErrorEnum.Unauthorized
println("Código: ${error.code}")
println("Mensaje: ${error.message}")
println("Mensaje completo: ${error.getFullMessage()}")

println("\n--- Todos los errores ---")
// Iterar sobre todos los valores
HttpErrorEnum.values().forEach { httpError ->
    println(httpError.getFullMessage())
}

Código: 401
Mensaje: No tienes permisos
Mensaje completo: Error 401: No tienes permisos

--- Todos los errores ---
Error 401: No tienes permisos
Error 404: Recurso no encontrado
Error 503: Servicio no disponible


## Sealed Classes (Clases Selladas)
Representan jerarquías de clases restringidas donde todos los subtipos son conocidos en tiempo de compilación.

In [3]:
// Sealed class para respuestas HTTP
sealed class HttpResponse {
    data class Success(val data: String, val statusCode: Int = 200): HttpResponse()
    data class Error(val message: String, val statusCode: Int): HttpResponse()
    object Loading: HttpResponse()
}

// Función para procesar respuestas
fun processResponse(response: HttpResponse): String = when(response) {
    is HttpResponse.Success -> "✅ Éxito (${response.statusCode}): ${response.data}"
    is HttpResponse.Error -> "❌ Error (${response.statusCode}): ${response.message}"
    HttpResponse.Loading -> "⏳ Cargando..."
}

// Ejemplos de uso
val responses = listOf(
    HttpResponse.Success("Datos del usuario obtenidos", 200),
    HttpResponse.Error("Usuario no encontrado", 404),
    HttpResponse.Loading,
    HttpResponse.Error("Servidor no disponible", 503)
)

responses.forEach { response ->
    println(processResponse(response))
}

✅ Éxito (200): Datos del usuario obtenidos
❌ Error (404): Usuario no encontrado
⏳ Cargando...
❌ Error (503): Servidor no disponible


In [4]:
// Sealed class más compleja para errores HTTP
sealed class HttpErrorClass(val code: Int) {
    data class Unauthorized(val reason: String): HttpErrorClass(401)
    object NotFound: HttpErrorClass(404)
    data class ServerUnavailable(val exception: String): HttpErrorClass(503)
    data class BadRequest(val validationErrors: List<String>): HttpErrorClass(400)
    
    // Función común para todas las subclases
    fun isServerError(): Boolean {
        return code >= 500
    }
    
    fun isClientError(): Boolean {
        return code in 400..499
    }
}

fun getErrorClass(error: HttpErrorClass): String = when(error) {
    is HttpErrorClass.Unauthorized -> "${error.code}. Usuario no autorizado. Razón: ${error.reason}"
    HttpErrorClass.NotFound -> "${error.code}. Recurso no encontrado"
    is HttpErrorClass.ServerUnavailable -> "${error.code}. Servicio no disponible. Excepción: ${error.exception}"
    is HttpErrorClass.BadRequest -> "${error.code}. Solicitud inválida. Errores: ${error.validationErrors.joinToString(", ")}"
}

// Ejemplos de errores
val errors = listOf(
    HttpErrorClass.Unauthorized("Token expirado"),
    HttpErrorClass.NotFound,
    HttpErrorClass.ServerUnavailable("Base de datos no disponible"),
    HttpErrorClass.BadRequest(listOf("Email inválido", "Contraseña muy corta"))
)

errors.forEach { error ->
    println(getErrorClass(error))
    println("¿Es error del servidor? ${error.isServerError()}")
    println("¿Es error del cliente? ${error.isClientError()}")
    println("---")
}

401. Usuario no autorizado. Razón: Token expirado
¿Es error del servidor? false
¿Es error del cliente? true
---
404. Recurso no encontrado
¿Es error del servidor? false
¿Es error del cliente? true
---
503. Servicio no disponible. Excepción: Base de datos no disponible
¿Es error del servidor? true
¿Es error del cliente? false
---
400. Solicitud inválida. Errores: Email inválido, Contraseña muy corta
¿Es error del servidor? false
¿Es error del cliente? true
---


## Sealed Interfaces
Similar a sealed classes, pero usando interfaces para mayor flexibilidad en la herencia.

In [ ]:
// Sealed interface para errores HTTP
sealed interface HttpErrorInterface {
    val timestamp: Long get() = System.currentTimeMillis()
    
    data class Unauthorized(val reason: String): HttpErrorInterface
    object NotFound: HttpErrorInterface
    data class ServerUnavailable(val exception: String): HttpErrorInterface
    data class RateLimited(val retryAfter: Int): HttpErrorInterface
}

// Interface adicional que pueden implementar algunos errores
interface Retryable {
    fun shouldRetry(): Boolean
    fun getRetryDelay(): Long
}

// Clase que implementa tanto la sealed interface como Retryable
data class NetworkTimeout(val timeoutDuration: Long) : HttpErrorInterface, Retryable {
    override fun shouldRetry(): Boolean = true
    override fun getRetryDelay(): Long = timeoutDuration * 2
}

fun getErrorInterface(error: HttpErrorInterface): String = when(error) {
    is HttpErrorInterface.Unauthorized -> "Usuario no autorizado. Razón: ${error.reason}"
    HttpErrorInterface.NotFound -> "Recurso no encontrado"
    is HttpErrorInterface.ServerUnavailable -> "Servicio no disponible. Razón: ${error.exception}"
    is HttpErrorInterface.RateLimited -> "Límite de requests excedido. Reintentar en ${error.retryAfter} segundos"
    is NetworkTimeout -> "Timeout de red después de ${error.timeoutDuration}ms"
}

// Función para manejar errores que pueden reintentarse
fun handleError(error: HttpErrorInterface) {
    println("Error: ${getErrorInterface(error)}")
    println("Timestamp: ${error.timestamp}")
    
    if (error is Retryable && error.shouldRetry()) {
        println("Se puede reintentar en ${error.getRetryDelay()}ms")
    } else {
        println("Error no recuperable")
    }
    println("---")
}

// Ejemplos
val interfaceErrors = listOf(
    HttpErrorInterface.Unauthorized("Sesión expirada"),
    HttpErrorInterface.NotFound,
    HttpErrorInterface.ServerUnavailable("Mantenimiento programado"),
    HttpErrorInterface.RateLimited(30),
    NetworkTimeout(5000)
)

interfaceErrors.forEach { error ->
    handleError(error)
}

## Diferencias claras: Sealed Class vs Sealed Interface
Aquí están las diferencias principales y cuándo usar cada uno.

In [ ]:
// Estado de una petición HTTP usando diferentes enfoques

// 1. Enum: Para estados fijos sin datos asociados
enum class RequestState {
    IDLE, LOADING, SUCCESS, ERROR
}

// 2. Siempre debe llevar un constructor
sealed class RequestResult {
    object Idle : RequestResult()
    object Loading : RequestResult()
    data class Success(val data: String, val headers: Map<String, String>) : RequestResult()
    data class Error(val exception: Throwable, val statusCode: Int?) : RequestResult()
}

// 3. No necesita contsructor
sealed interface ApiResult {
    data class Success(val data: String) : ApiResult
    data class Error(val exception: Throwable) : ApiResult
    object Loading : ApiResult
}

## Ejemplo completo: Sistema de autenticación HTTP
Combinando todos los conceptos en un ejemplo práctico.

In [ ]:
// Enum para roles de usuario
enum class UserRole(val permissions: List<String>) {
    ADMIN(listOf("read", "write", "delete", "admin")),
    MODERATOR(listOf("read", "write", "moderate")),
    USER(listOf("read", "write")),
    GUEST(listOf("read"));

    fun canPerform(action: String): Boolean {
        return permissions.contains(action)
    }
}

// Sealed class para resultado de autenticación
sealed class AuthResult {
    data class Success(val token: String, val user: AuthUser) : AuthResult()
    data class Failed(val reason: AuthError) : AuthResult()
    object InProgress : AuthResult()
}

// Sealed interface para errores de autenticación
sealed interface AuthError {
    val message: String

    data class InvalidCredentials(override val message: String = "Usuario o contraseña incorrectos") : AuthError
    data class AccountLocked(val unlockTime: Long, override val message: String = "Cuenta bloqueada") : AuthError
    data class NetworkError(val exception: Exception, override val message: String = "Error de conexión") : AuthError
    object ServerError : AuthError {
        override val message: String = "Error interno del servidor"
    }
}

// Data class para usuario
data class AuthUser(
    val id: Int,
    val username: String,
    val email: String,
    val role: UserRole
)

// Sistema de autenticación
class AuthSystem {

    fun authenticate(username: String, password: String): AuthResult {
        return when {
            username.isEmpty() || password.isEmpty() ->
                AuthResult.Failed(AuthError.InvalidCredentials())

            username == "admin" && password == "admin123" -> {
                val user = AuthUser(1, "admin", "admin@sistema.com", UserRole.ADMIN)
                AuthResult.Success("token_admin_123", user)
            }

            username == "user" && password == "user123" -> {
                val user = AuthUser(2, "user", "user@sistema.com", UserRole.USER)
                AuthResult.Success("token_user_456", user)
            }

            username == "locked" ->
                AuthResult.Failed(AuthError.AccountLocked(System.currentTimeMillis() + 300000))

            else -> AuthResult.Failed(AuthError.InvalidCredentials())
        }
    }

    fun handleAuthResult(result: AuthResult) {
        when (result) {
            is AuthResult.Success -> {
                println("✅ Autenticación exitosa")
                println("   Usuario: ${result.user.username}")
                println("   Rol: ${result.user.role}")
                println("   Permisos: ${result.user.role.permissions.joinToString(", ")}")
                println("   Token: ${result.token}")

                // Probar permisos
                val actions = listOf("read", "write", "delete", "admin")
                actions.forEach { action ->
                    val canPerform = result.user.role.canPerform(action)
                    val status = if (canPerform) "✅" else "❌"
                    println("   $status Puede realizar: $action")
                }
            }

            is AuthResult.Failed -> {
                println("❌ Autenticación fallida")
                when (val error = result.reason) {
                    is AuthError.InvalidCredentials ->
                        println("   Razón: ${error.message}")

                    is AuthError.AccountLocked -> {
                        println("   Razón: ${error.message}")
                        println("   Se desbloqueará en: ${error.unlockTime - System.currentTimeMillis()}ms")
                    }

                    is AuthError.NetworkError -> {
                        println("   Razón: ${error.message}")
                        println("   Excepción: ${error.exception.message}")
                    }

                    AuthError.ServerError ->
                        println("   Razón: ${error.message}")
                }
            }

            AuthResult.InProgress ->
                println("⏳ Autenticando...")
        }
        println()
    }
}

// Pruebas del sistema
val authSystem = AuthSystem()

val testCredentials = listOf(
    Pair("admin", "admin123"),
    Pair("user", "user123"),
    Pair("guest", "wrong"),
    Pair("locked", "password"),
    Pair("", "")
)

testCredentials.forEach { (username, password) ->
    println("=== Probando: $username / $password ===")
    val result = authSystem.authenticate(username, password)
    authSystem.handleAuthResult(result)
}